In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import time
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import sqlite3


db_name = 'redbus_data.db'

def insert_into_table(bus_data,cursor,conn):
    sql = """
    INSERT INTO Bus_Routes (route_name, route_link, bus_name, bus_type, departure_time, duration, arrival_time, rating, price, availability)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """
    values = (
        bus_data['route_name'], bus_data['route_link'], bus_data['bus_name'], bus_data['bus_type'],
        bus_data['departure_time'], bus_data['duration'], bus_data['arrival_time'],
        bus_data['rating'], bus_data['price'], bus_data['availability']
    )
    cursor.execute(sql, values)
    conn.commit()


def get_bus_links(driver, urls):
    bus_link = []
    for url in urls:
        driver.get(url)
        time.sleep(1)
        page_tabs = driver.find_elements(By.CLASS_NAME, 'DC_117_pageTabs')
        # print(page_tabs)
        links = 0
        for page_tab in page_tabs:            
            try:
                
                driver.execute_script("arguments[0].click();", page_tab)
                time.sleep(.2)
                Redbus_Links = driver.find_elements(By.CLASS_NAME, 'route')
                links += len(Redbus_Links)
                for busroute_link in Redbus_Links:
                    link = busroute_link.get_attribute('href')
                    if link and link.startswith("https://www.redbus.in/bus-tickets/"):
                        bus_link.append(link)
            except Exception as e:
                print(f"Error clicking page tab {page_tab.text}: {e}")
            print(f"Found {(links)} footer links on {url}")   
    return bus_link


def create_table():
    try:
        conn = sqlite3.connect(db_name)
        cursor = conn.cursor()
        table_schema = """
        CREATE TABLE IF NOT EXISTS Bus_Routes (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            route_name VARCHAR(255),
            route_link VARCHAR(255),
            bus_name VARCHAR(255),
            bus_type VARCHAR(255),
            departure_time VARCHAR(255),
            duration VARCHAR(255),
            arrival_time VARCHAR(255),
            rating FLOAT,
            price VARCHAR(255),
            availability VARCHAR(255),
            CONSTRAINT Filter Unique('route_name','bus_name','departure_time')
        );
        """
        cursor.execute(table_schema)
        conn.commit()
    except Exception as e:
        print(e)
    finally:
        conn.close()

def drop_table():
    try:
        conn = sqlite3.connect(db_name)
        cursor = conn.cursor()
        table_schema = """
        DROP TABLE IF EXISTS Bus_Routes;
        """
        cursor.execute(table_schema)
        conn.commit()
    except Exception as e:
        print(e)
    finally:
        conn.close()

        
def scrape_bus_data(driver, bus_link):
    try:
        conn = sqlite3.connect(db_name)
        cursor = conn.cursor()
        bus_data = []

        for i in bus_link:
            try:
                print(i)
                driver.get(i)
                WebDriverWait(driver, 40).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'row-sec.clearfix')))
                pages = 1
                while 100>pages:
                    buses = len(driver.find_elements(By.CLASS_NAME, 'row-sec.clearfix')) 
                    # print(buses)
                    pages+=1   
                    driver.execute_script("window.scrollBy(0, 500)")    
                    
                buses = driver.find_elements(By.CLASS_NAME, 'row-sec.clearfix')
                print(f"Found {len(buses)} buses on page {i}")
                
                for bus in buses:
                    try:
                        bus_name = bus.find_element(
                            By.CLASS_NAME, 'travels.lh-24.f-bold.d-color').text.strip()
                        bus_type = bus.find_element(
                            By.CLASS_NAME, 'bus-type.f-12.m-top-16.l-color.evBus').text.strip()
                        # departure_time = bus.find_element(
                        #     # dp-time f-19 d-color f-bold
                        #     By.CLASS_NAME, 'column-three.p-right-10.w-10.fl').text.strip()
                        departure_time = bus.find_element(
                            By.CLASS_NAME, 'dp-time.f-19.d-color.f-bold').text.strip()

                        # duration = bus.find_element(
                        #     By.CLASS_NAME, 'column-five.p-right-10.w-10.fl').text.strip()
                        duration = bus.find_element(
                            By.CLASS_NAME, 'dur.l-color.lh-24').text.strip()
                        # arrival_time = bus.find_element(
                        #     By.CLASS_NAME, 'column-five.p-right-10.w-10.fl').text.strip()
                        arrival_time = bus.find_element(
                            By.CLASS_NAME, 'bp-time.f-19.d-color.disp-Inline').text.strip()
                        try:
                            rating = bus.find_element(
                                By.CLASS_NAME, 'rating-sec.lh-24').text.strip()
                        except:
                            rating = 0
                        price = float(bus.find_element(
                            By.CLASS_NAME, 'fare.d-block').text.strip().lower().replace('inr ', ''))

                        try:
                            availability = bus.find_element(
                                By.CLASS_NAME, 'seat-left.m-top-30').text.strip()
                        except:
                            availability = bus.find_element(
                                By.CLASS_NAME, 'seat-left.m-top-16').text.strip()

                        if bus_name and price:  # Ensure critical elements are found
                            bus_data = {
                                'route_name': i[34:],
                                'route_link': i,
                                'bus_name': bus_name,
                                'bus_type': bus_type,
                                'departure_time': departure_time,
                                'duration': duration,
                                'arrival_time': arrival_time,
                                'rating': rating,
                                'price': price,
                                'availability': availability
                            }
                            insert_into_table(bus_data,cursor,conn)
                            print(f"Added bus: {bus_name}")
                        else:
                            print(f"Skipping incomplete data for bus on page {i}")

                    except Exception as e:
                        print(f"Error scraping bus data: {e}")

            except Exception as e:
                print(f"Error loading page {i}: {e}")
    except Exception as e:
        print(e)
    finally:
        conn.close()

    return bus_data


driver = webdriver.Chrome()

urls = [
    'https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile',
    'https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile',
    'https://www.redbus.in/online-booking/tsrtc/?utm_source=rtchometile',
    'https://www.redbus.in/online-booking/ktcl/?utm_source=rtchometile',
    'https://www.redbus.in/online-booking/rsrtc/?utm_source=rtchometile',
    'https://www.redbus.in/online-booking/south-bengal-state-transport-corporation-sbstc/?utm_source=rtchometile',
    'https://www.redbus.in/online-booking/hrtc/?utm_source=rtchometile',
    'https://www.redbus.in/online-booking/astc/?utm_source=rtchometile',
    'https://www.redbus.in/online-booking/uttar-pradesh-state-road-transport-corporation-upsrtc/?utm_source=rtchometile',
    'https://www.redbus.in/online-booking/chandigarh-transport-undertaking-ctu',
    'https://www.redbus.in/online-booking/west-bengal-transport-corporation',
    'https://www.redbus.in/online-booking/north-bengal-state-transport-corporation',
    'https://www.redbus.in/online-booking/meghalaya-transport-corporation-mtc'
]

try:
    bus_link = get_bus_links(driver, urls)
    print(f"Collected {len(bus_link)} bus links")

    if bus_link:
        choice = int(input('Press 1 to recreate db, Press 2 to append to old db'))
        if choice == 1:
            drop_table()
            create_table()
            bus_data = scrape_bus_data(driver, bus_link)
        elif choice == 2:
            bus_data = scrape_bus_data(driver, bus_link)
        print(bus_data)
    else:
        print("No bus links found.")

except Exception as e:
    print(f"Unexpected error: {e}")

finally:
    driver.quit()

Found 10 footer links on https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile
Found 20 footer links on https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile
Found 30 footer links on https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile
Found 40 footer links on https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile
Found 49 footer links on https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile
Found 10 footer links on https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile
Found 19 footer links on https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile
Found 10 footer links on https://www.redbus.in/online-booking/tsrtc/?utm_source=rtchometile
Found 20 footer links on https://www.redbus.in/online-booking/tsrtc/?utm_source=rtchometile
Found 26 footer links on https://www.redbus.in/online-booking/tsrtc/?utm_source=rtchometile
Found 10 footer links on https://www.redbus.in/online-booking

In [10]:
%%writefile applicationsam.py
import streamlit as st
import pandas as pd
import sqlite3
from sqlalchemy import create_engine



css='''
<style>
    [data-testid="stExpander"] div:has(>.streamlit-expanderContent) {
        overflow: scroll;
        height: 400px;
    }
</style>
'''

st.markdown(css, unsafe_allow_html=True)

db_name = 'redbus_data.db'

def make_clickable(val):
    return f'<a target="_blank" href="{val}">{val}</a>'


def load_data():
    conn = sqlite3.connect(db_name)
    query = "SELECT * FROM Bus_Routes"
    df = pd.read_sql(query, conn)
    conn.close()
    return df


df = load_data()


df['price'] = pd.to_numeric(df['price'], errors='coerce')


st.title('Bus Route Finder')


bus_types = st.multiselect('Select Bus Type', options=df['bus_type'].unique())
routes = st.multiselect('Select Route', options=df['route_name'].unique())
price_range = st.slider('Select Price Range', min_value=int(df['price'].min()), max_value=int(df['price'].max()), value=(int(df['price'].min()), int(df['price'].max())))
star_rating = st.slider('Select Star Rating', min_value=float(df['rating'].min()), max_value=float(df['rating'].max()), value=(float(df['rating'].min()), float(df['rating'].max())))
availability = st.selectbox('Select Availability', options=['All', 'Available'])


filtered_df = df.copy()
if bus_types:
    filtered_df = filtered_df[filtered_df['bus_type'].isin(bus_types)]
if routes:
    filtered_df = filtered_df[filtered_df['route_name'].isin(routes)]
filtered_df = filtered_df[(filtered_df['price'] >= price_range[0]) & (filtered_df['price'] <= price_range[1])]
filtered_df = filtered_df[(filtered_df['rating'] >= star_rating[0]) & (filtered_df['rating'] <= star_rating[1])]
if availability == 'Available':
    filtered_df = filtered_df[filtered_df['availability'] != 'Sold Out']


filtered_df['route_link'] = filtered_df['route_link'].apply(make_clickable)

st.markdown(filtered_df[['bus_name', 'route_link', 'price', 'availability']].to_html(escape=False), unsafe_allow_html=True)


#st.dataframe(filtered_df[['bus_name','route_link','price','availability']])


if st.checkbox('Show Raw Data'):
    st.write(df)

Overwriting applicationsam.py


In [11]:
!streamlit run applicationsam.py

^C


In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import time
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import sqlite3


conn = sqlite3.connect('redbus_data.db')
cursor = conn.cursor()

cursor.execute("select count(*) from bus_routes")

for i in cursor:
    print(i)

(5635,)
